In [1]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [2]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [3]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb", 'results.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [4]:
parcels_centers = pd.DataFrame.spatial.from_featureclass(r".\Inputs\ultimate_parcel_eq.gdb\parcels_centers")
parcels_infill = pd.DataFrame.spatial.from_featureclass(r".\Inputs\ultimate_parcel_eq.gdb\parcels_infill")
parcels_counties = pd.DataFrame.spatial.from_featureclass(r".\Inputs\ultimate_parcel_eq.gdb\parcels_counties")
parcels_cityarea = pd.DataFrame.spatial.from_featureclass(r".\Inputs\ultimate_parcel_eq.gdb\parcels_cityarea")

In [5]:
parcels_centers2 = parcels_centers[['parcel_id', 'TAZID_832','TAZID_900', 'AreaName',	'AreaType']].copy()
parcels_centers2.rename({'AreaName': 'CENTER_NAME','AreaType': 'CENTER_TYPE'}, axis=1, inplace=True)

parcels_infill2 = parcels_infill[['parcel_id', 'Join_Count']].copy()
parcels_infill2.rename({'Join_Count': 'INFILL1990'}, axis=1, inplace=True)

parcels_counties2 = parcels_counties[['parcel_id', 'NAME', 'FIPS']].copy()
parcels_counties2.rename({'NAME': 'COUNTY_NAME', 'FIPS': 'COUNTY_FIPS'}, axis=1, inplace=True)
parcels_counties2.loc[parcels_counties2['COUNTY_FIPS'].isin([49])==False , 'MPO'] = 'WFRC'
parcels_counties2.loc[parcels_counties2['COUNTY_FIPS'].isin([49])==True , 'MPO'] = 'MAG'

parcels_cityarea2 = parcels_cityarea[['parcel_id', 'CityArea']].copy()
parcels_cityarea2.rename({'CityArea': 'CITY_AREA'}, axis=1, inplace=True)

In [6]:
parcel_eq = parcels_centers2.merge(parcels_cityarea2, on='parcel_id', how='left').merge(parcels_counties2, on='parcel_id', how='left').merge(parcels_infill2, on='parcel_id', how='left')
parcel_eq.head()

,parcel_id,TAZID_832,TAZID_900,CENTER_NAME,CENTER_TYPE,CITY_AREA,COUNTY_NAME,COUNTY_FIPS,MPO,INFILL1990
0,1,611,832,Bountiful Downtown,City Center,Bountiful,DAVIS,11.0,WFRC,1
1,2,527,704,Layton Midtown,Urban Center,Layton,DAVIS,11.0,WFRC,0
2,3,620,867,North Salt Lake / Woods Cross Industrial District,Industrial District,Woods Cross,DAVIS,11.0,WFRC,0
3,4,619,862,None,None,Woods Cross,DAVIS,11.0,WFRC,0
4,5,605,825,Gateway Crossing,Neighborhood Center,Bountiful,DAVIS,11.0,WFRC,1


In [7]:
parcel_eq.to_csv(r'.\Outputs\parcel_eq_v4.csv', index=False)